<a href="https://colab.research.google.com/github/gdcc/easyDataverse/blob/main/examples/EasyDataverseBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EasyDataverse Basics

Welcome to this introductory notebook that demonstrates the basic features of EasyDataverse. EasyDataverse is a user-friendly tool that aims to simplify the process of managing data with Dataverse. The tool utilizes Dataverse's API-first philosophy and offers a GUI-like experience when interacting with datasets. It is backed by Dataverse's powerful Native API, providing an efficient, streamlined, and simplified data management solution.

Within this notebook you will learn more about the following topics:

* Connecting to a Dataverse instance
* Gathering metadata block information
* Creating a dataset and adding metadata
* File uploads (S3 and native)
* Fetching and updating datasets

In [ ]:
# Install easyDataverse
!pip install easyDataverse

# Download data
!wget https://github.com/gdcc/easyDataverse/raw/main/examples/data/README.md
!wget https://github.com/gdcc/easyDataverse/raw/main/examples/data/anscomb.json
!wget https://github.com/gdcc/easyDataverse/raw/main/examples/data/mnist_test.csv
!wget https://github.com/gdcc/easyDataverse/raw/main/examples/data/mnist_train_small.csv 

In [1]:
import rich # For printing

from easyDataverse import Dataverse

### Connecting to Dataverse

To begin, you can establish a connection to a Dataverse by using the `Dataverse`the connection process, EasyDataverse will gather all available metadata blocks and convert them into PyDantic data types. These data types follow a strict typing system to ensure that any data passed in complies with the expectations of your instance. These types will come in handy later when creating datasets.

In [4]:
# Connect to a Dataverse instance of choice
dataverse = Dataverse(
    server_url='https://demo.dataverse.org', # @param {type:"string"}
    api_token='<API_TOKEN>' # @param {type:"string"}
)

Output()

🎉 Connected to 'https://demo.dataverse.org/'

### Metadata block fields

It's possible that you might not be familiar with all the metadata fields that are available, but you don't want to keep switching back and forth between the Graphical User Interface (GUI) and the notebook. Fortunately, EasyDataverse offers a solution to this problem by providing a `list_metadatablocks` method that allows you to check what metadata blocks are available. This method has two modes of functionality:

* `detailed=False` - Lists the names of the available metadata blocks
* `detailed=True` - Lists each metadata block, its fields and available add-methods

In [5]:
# Mode 1: Only names
dataverse.list_metadatablocks(detailed=False)

# Mode 2: Names, fields and methods 
# dataverse.list_metadatablocks(detailed=True)

geospatial
socialscience
astrophysics
biomedical
journal
customMRA
customGSD
customARCS
customPSRI
customPSI
customCHIA
customDigaai
citation
customSAEF


----

### Dataset creation and population

Our next step is to establish a new dataset and add relevant metadata and files to it. Once we connect to the Dataverse instance, we can use the `dataverse` object that we had previously created to generate new datasets. The `create_dataset` method will return a `Dataset` object that comes with all the available metadata blocks from the connected instance.

In [63]:
dataset = dataverse.create_dataset()

print(dataset) # Should be empty by now

metadatablocks: {}



Before we populate the metadata of this dataset, we should check what information is available. Each metadata block can be accessed as an attribute of the `dataset` instance. Just like we can check the content of all metadata blocks, we can also inspect the content of individual instances. Specifically, we will inspect the content of the `geospatial` metadata block.

In [64]:
dataset.geospatial.info()

Geospatial
├── unit: str
├── coverage: GeographicCoverage
│   ├── country
│   ├── state
│   ├── city
│   ╰── other_geographic_coverage
├── bounding_box: GeographicBoundingBox
│   ├── west_longitude
│   ├── east_longitude
│   ├── north_latitude
│   ╰── south_latitude
╰── Add Functions
    ├── add_coverage
    ╰── add_bounding_box

Once we have a clear understanding of the fields available, we can add values to the corresponding attributes/fields in the metadata block. For lists of compound objects or sub-objects, we can add them using dedicated add-methods that will be demonstrated below.

In [65]:
# Singular fields
dataset.geospatial.unit = ["unit"]

# Adding to list of compounds
dataset.geospatial.add_coverage(
    country="Germany",
    state="BW",
    city="Stuttgart",
)

print(dataset)

metadatablocks:
  geospatial:
    geographicUnit:
      - unit
    geographicCoverage:
      - country: Germany
        state: BW
        city: Stuttgart



### Data validation

You may have noticed that instead of setting a single unit, we have provided a list of units to the `unit` field. This is because the metadata block expects this field to be a multiple of text. As the input we supplied meets this requirement, we did not encounter any problems. Similarly, the `country` field of the added `coverage` compound also expects a value from a controlled vocabulary.

EasyDataverse provides data types that follow a strict typing system. This system is enforced during initialization and assignment, which helps to prevent errors during upload and enables developers to identify non-compliant data. Let's attempt to intentionally provide incorrect input types and see how the system responds.

In [66]:
# Using try-except to not break the flow
from pydantic import ValidationError

try:
  # Adding a non-list element
  dataset.geospatial.unit = "unit"
except ValidationError as e:
  rich.print(e)

try:
  # Adding a value that is not part of a controlled vocab
  dataset.geospatial.add_coverage(
      country="germany",
      state="BW",
      city="Stuttgart",
  )
except ValidationError as e:
  # ... you see the world is quite large :')
  rich.print(e)

1 validation error for Geospatial
unit
  Input should be a valid list [type=list_type, input_value='unit', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/list_type

1 validation error for GeographicCoverage
country
  Input should be 'Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 
'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 
'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 
'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 
'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 
'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 
'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, the 
Democratic Republic of the', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech 
Republic', "Côte d'Ivoire", 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El 
Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 
'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia',
'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 
'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and Mcdonald Islands', 'Holy See (Vatican 
City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 
'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya',
'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's 
Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 
'Luxembourg', 'Macao', 'Macedonia, the Former Yugoslav Republic of', 'Madagascar', 'Malawi', 'Malaysia', 
'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 
'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 
'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 
'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 
'Pakistan', 'Palau', 'Palestine, State of', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 
'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Romania', 'Russian Federation', 'Rwanda', 'Réunion', 
'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint 
Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao
Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten 
(Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South 
Sandwich Islands', 'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'Svalbard and Jan Mayen', 'Swaziland',
'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United 
Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 
'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United 
Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, 
Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 
'Western Sahara', 'Yemen', 'Zambia', 'Zimbabwe' or 'Åland Islands' [type=enum, input_value='germany', 
input_type=str]
    For further informatio

To proceed, we can add more information to our dataset by including metadata. For now, we will only focus on filling out the required fields, but don't hesitate to try out other metadata options. It's worth noting that EasyDataverse will check for the mandatory fields during the upload process.

In [67]:
dataset.citation.title = "My Dataset"
dataset.citation.add_author(name="John Doe")
dataset.citation.add_dataset_contact(name="John Doe", email="john@doe.com")
dataset.citation.add_ds_description(value="This is a description")
dataset.citation.subject = ["Other"]

### File uploads

To add files to a dataset, you can utilize dedicated methods to add either a single file or a directory of files. Both of these methods are available within the `Dataset` instance and are register your files for upload. You also have the option to customize the directory in which the file is stored and rename it according to your preferences.

In [68]:
# Upload a single file
dataset.add_file(
    local_path="./data/anscomb.json", # Path to the file on your system
    dv_dir="some/sub/dir", # Optional sub directory on Dataverse
    file_name="different_name.json" # Optional renaming of the file
)


# Upload an entire directory
dataset.add_directory(
    dirpath="./data",
    ignores=[
        r"anscomb.json", # Ignore the previously added file
        "^\..*",         # Ignore hidden files and dirs
    ]
)

# Check if all the files have been added
rich.print(f"Added {len(dataset.files)} files.")

Added 4 files.

### Uploading a dataset

Before we upload a dataset, let's take a look at what goes on behind the scenes. EasyDataverse creates a Dataverse JSON that adds metadata to your dataset, which can be viewed by calling the `dataverse_dict`-method. However, you don't need to worry about this as EasyDataverse takes care of it for you.

In [69]:
rich.print(dataset.dataverse_dict())

{
    'datasetVersion': {
        'metadataBlocks': {
            'geospatial': {
                'fields': [
                    {'multiple': True, 'typeClass': 'primitive', 'typeName': 'geographicUnit', 'value': ['unit']},
                    {
                        'multiple': True,
                        'typeClass': 'compound',
                        'typeName': 'geographicCoverage',
                        'value': [
                            {
                                'country': {
                                    'multiple': False,
                                    'typeClass': 'controlledVocabulary',
                                    'typeName': 'country',
                                    'value': 'Germany'
                                },
                                'state': {
                                    'multiple': False,
                                    'typeClass': 'primitive',
                                    'typeName': 'state',
                                    'value': 'BW'
                                },
                                'city': {
                                    'multiple': False,
                                    'typeClass': 'primitive',
                                    'typeName': 'city',
                                    'value': 'Stuttgart'
                                }
                            }
                        ]
                    }
                ]
            },
            'citation': {
                'fields': [
                    {'multiple': False, 'typeClass': 'primitive', 'typeName': 'title', 'value': 'My Dataset'},
                    {
                        'multiple': True,
                        'typeClass': 'controlledVocabulary',
                        'typeName': 'subject',
                        'value': ['Other']
                    },
                    {
                        'multiple': True,
                        'typeClass': 'compound',
                        'typeName': 'author',
                        'value': [
                            {
                                'authorName': {
                                    'multiple': False,
                                    'typeClass': 'primitive',
                                    'typeName': 'authorName',
                                    'value': 'John Doe'
                                }
                            }
                        ]
                    },
                    {
                        'multiple': True,
                        'typeClass': 'compound',
                        'typeName': 'datasetContact',
                        'value': [
                            {
                                'datasetContactName': {
                                    'multiple': False,
                                    'typeClass': 'primitive',
                                    'typeName': 'datasetContactName',
                                    'value': 'John Doe'
                                },
                                'datasetContactEmail': {
                                    'multiple': False,
                                    'typeClass': 'primitive',
                                    'typeName': 'datasetContactEmail',
                                    'value': 'john@doe.com'
                                }
                            }
                        ]
                    },
                    {
                        'multiple': True,
                        'typeClass': 'compound',
                        'typeName': 'dsDescription',
                        'value': [
                            {
                                'dsDescriptionValue': {
                                    'multiple': False,
                                    'typeClass': 'primitive',
                                    'typeName': 'dsDescriptionValue',
                 

After completing all the necessary steps, we are now ready to move on to the final step which is to upload the dataset to Demo Dataverse.

> Please note that this collection is privat and you may want to replace it with one of your own. Also, this is an S3-enabled collection that demonstrates how you can directly upload to S3!

In [70]:
pid = dataset.upload(dataverse_name="ed_test", n_parallel=4)

Dataset with pid 'doi:10.70122/FK2/2ZNL2G' created.




╭──────────── DVUploader ─────────────╮
│ Server: https://demo.dataverse.org/ │
│ PID: doi:10.70122/FK2/2ZNL2G        │
│ Files: 4                            │
╰─────────────────────────────────────╯

Output()

🔎 Checking dataset files                  
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ File                  ┃ Status ┃ Action ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ different_name.json   │ New    │ Upload │
│ README.md             │ New    │ Upload │
│ mnist_train_small.csv │ New    │ Upload │
│ mnist_test.csv        │ New    │ Upload │
└───────────────────────┴────────┴────────┘

🚀 Uploading files

Output()

✅ Upload complete

---

## Fetching and updating datasets

To fetch datasets, you can use the `load_dataset` method of a connected `Dataverse` instance. However, you must provide the persistent identifier (such as a DOI) of the dataset for this to work. Once you have provided the identifier, the method will automatically download all files of the dataset and restore the metadata block objects, just like the previous dataset we have created.

In [72]:
dataset = dataverse.load_dataset(pid)

print(dataset)

Fetching dataset 'doi:10.70122/FK2/2ZNL2G' from 'https://demo.dataverse.org/'

╭─ Dataset Information ─╮
│ Title: My Dataset     │
│ Version: latest       │
│ Files: 4              │
╰───────────────────────╯

Output()

Downloading files

╰── ✅ Done 

metadatablocks:
  geospatial:
    geographicUnit:
      - unit
    geographicCoverage:
      - country: Germany
        state: BW
        city: Stuttgart
  citation:
    title: My Dataset
    subject:
      - Other
    author:
      - authorName: John Doe
    datasetContact:
      - datasetContactName: John Doe
        datasetContactEmail: john@doe.com
    dsDescription:
      - dsDescriptionValue: This is a description
dataset_id: doi:10.70122/FK2/2ZNL2G



### Updating a dataset

After you have fetched a dataset and you want to edit it, you can simply edit the metadata and/or add files to the dataset. The files Once you have retrieved a dataset and wish to make changes to it, you can easily modify the metadata or append additional files to the dataset. Additionally, you can alter the existing files that were downloaded during the initial retrieval process. The uploader will detect any changes you make and prompt a replacement upload. However, if a file remains unchanged, it will not be uploaded again.

In [73]:
dataset.citation.title = "My updated dataset"

dataset.update()

╭──────────── DVUploader ─────────────╮
│ Server: https://demo.dataverse.org/ │
│ PID: doi:10.70122/FK2/2ZNL2G        │
│ Files: 4                            │
╰─────────────────────────────────────╯

Output()

🔎 Checking dataset files                     
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓
┃ File                  ┃ Status    ┃ Action ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩
│ README.md             │ Same hash │ Skip   │
│ different_name.json   │ Same hash │ Skip   │
│ mnist_test.csv        │ Same hash │ Skip   │
│ mnist_train_small.csv │ Same hash │ Skip   │
└───────────────────────┴───────────┴────────┘

❌ No files to upload